In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

station = pd.read_csv("./094/station.csv", sep='\t', engine="python")
patient = pd.read_csv("./094/patient.csv", sep='\t', engine="python")
observation = pd.read_csv("./094/observation.csv", sep='\t', engine="python")

print("patient.shape =", patient.shape)
print("station.shape =", station.shape)
print("observation.shape =", observation.shape)

### 1.1 Základný opis dát spolu s ich charakteristikami 

### (A-1b)

In [ ]:
# (A-1b)
print("patient columns:", patient.columns.tolist())
print("station columns:", station.columns.tolist())
print("observation columns:", observation.columns.tolist())

#### 🩺 Súbor patient.csv
- **Počet záznamov:** 2 102  
- **Počet atribútov:** 13  
- **Typy dát:** object = 10, int64 = 2, float64 = 1  
- **Chýbajúce hodnoty:** spolu ≈ 3 993  
  - Najviac chýba: `residence` (100 %), `job` (70 %), `address` (15 %), `current_location` (5 %).  
- **Charakteristika:** obsahuje demografické údaje pacientov a odkaz na stanicu (`station_ID`).  
  Tento odkaz sa **nedá priamo spárovať** s názvom v `súbore station.csv`.  

#### ⚙️ Súbor station.csv
- **Počet záznamov:** 798  
- **Počet atribútov:** 6  
- **Typy dát:** object = 4, float64 = 2  
- **Chýbajúce hodnoty:** 0  
- **Charakteristika:** obsahuje informácie o meracích staniciach – `station`, `latitude`, `longitude`, `QoS`, `revision`, `location`.  
- **Pozorovanie:** hodnoty `revision` majú rôzne formáty dátumov (a časť nevieme správne parsovať) → potrebná normalizácia na jednotný formát `datetime`.

#### 📊 Súbor observation.csv
- **Počet záznamov:** 12 081  
- **Počet atribútov:** 23  
- **Typy dát:** všetky `float64`  
- **Chýbajúce hodnoty:** 0  
- **Cieľová premenná:** `oximetry` (binárna 0/1).  
- **Dôležité atribúty:** `SpO₂`, `HR`, `Skin Temperature`, `BP`, `CO`, `FiO₂`, atď.  
- Hodnoty SpO₂ sú v rozsahu 95 – 100 a Skin Temperature v rozsahu 33 – 38 °C.  
- Tieto dáta majú vhodný formát pre ďalšie spracovanie v Python/pandas a na trénovanie modelov.


In [ ]:
def dtype_counts(df): 
    return df.dtypes.astype(str).value_counts().to_dict()

summ = {
    "patient": {
        "shape": patient.shape,
        "dtype_counts": dtype_counts(patient),
        "missing_total": int(patient.isna().sum().sum())
    },
    "station": {
        "shape": station.shape,
        "dtype_counts": dtype_counts(station),
        "missing_total": int(station.isna().sum().sum())
    },
    "observation": {
        "shape": observation.shape,
        "dtype_counts": dtype_counts(observation),
        "missing_total": int(observation.isna().sum().sum())
    }
}
summ


#### Analýza chýbajúcich hodnôt (EDA)


In [ ]:
missing_pct = (patient.isna().sum() / len(patient) * 100).round(2).sort_values(ascending=False)
missing_pct.head(10)

#### Z výpočtu percenta chýbajúcich hodnôt vidíme, že niektoré atribúty obsahujú výrazný počet prázdnych záznamov:

#### Atribút	Podiel chýbajúcich hodnôt
residence	100 %
job	≈ 70 %
address	≈ 15 %
current_location	≈ 5 %

#### 🔗 Vzťahy medzi súbormi
| Vzťah | Typ väzby | Popis |
|:--|:--:|:--|
| observation ↔ station | 1 : N | Každé meranie má priradenú stanicu (100 % zhoda cez `latitude`, `longitude`). |
| patient ↔ station | ? | Nepodarilo sa spárovať cez `current_location`, zhoda 0 %. |
| patient ↔ observation | – | Chýba priame prepojenie (pacient ID v meraniach neexistuje). |

In [ ]:
sta_lat = pd.to_numeric(station["latitude"], errors="coerce").round(4)
sta_lon = pd.to_numeric(station["longitude"], errors="coerce").round(4)
obs_lat = pd.to_numeric(observation["latitude"], errors="coerce").round(4)
obs_lon = pd.to_numeric(observation["longitude"], errors="coerce").round(4)

station_key = set(zip(sta_lat, sta_lon))
obs_key = list(zip(obs_lat, obs_lon))

obs_valid = sum([not (pd.isna(k[0]) or pd.isna(k[1])) for k in obs_key])
obs_match = sum([(k in station_key) for k in obs_key if not (pd.isna(k[0]) or pd.isna(k[1]))])

print(f"observation → station match by coords: {obs_match}/{obs_valid} ({obs_match/obs_valid:.3f})")

# patient ↔ station:
import re
def parse_current_location(s):
    if not isinstance(s, str): 
        return (np.nan, np.nan)
    nums = re.findall(r"Decimal\\('([-+]?\\d*\\.?\\d+)'\\)", s)
    return (float(nums[0]), float(nums[1])) if len(nums)==2 else (np.nan, np.nan)

latlon = patient["current_location"].apply(parse_current_location)
pat_lat = pd.to_numeric([t[0] for t in latlon], errors="coerce").round(4)
pat_lon = pd.to_numeric([t[1] for t in latlon], errors="coerce").round(4)

pat_valid = np.isfinite(pat_lat).sum()
pat_match = sum([(k in station_key) for k in zip(pat_lat, pat_lon) if not (pd.isna(k[0]) or pd.isna(k[1]))])

print(f"patient → station match by coords: {pat_match}/{pat_valid} ({(pat_match/pat_valid if pat_valid else 0):.3f})")


#### 📈 Vizualizácia (EDA)
Histogramy rozloženia hodnôt:
- **SpO₂** → zväčša v rozmedzí 97 – 99 %  
- **Skin Temperature** → rozloženie 33 – 38 °C s vrcholom okolo 36 °C  

Tieto vizualizácie potvrdzujú, že dáta majú správny a očakávaný fyzio-rozsah.


In [ ]:
import matplotlib.pyplot as plt

# 1) SpO2
plt.figure()
observation["SpO₂"].hist(bins=30, edgecolor="black")
plt.title("Rozloženie SpO₂")
plt.xlabel("SpO₂")
plt.ylabel("Frekvencia")
plt.show()

# 2) Skin Temperature
plt.figure()
observation["Skin Temperature"].hist(bins=30, edgecolor="black")
plt.title("Rozloženie Skin Temperature")
plt.xlabel("°C")
plt.ylabel("Frekvencia")
plt.show()


### 🧩 Zhrnutie zistení
- Dáta majú zrozumiteľnú štruktúru a sú v správnom formáte na ďalšie spracovanie.  
- Najviac problémov má `súbor patient.csv` → mnoho chýbajúcich hodnôt.  
- `súbor station.csv` má nekonzistentné formáty v atribúte `revision`.  
- `súbor observation.csv` je čistý a vhodný na modelovanie (strojové učenie).  
- Vzťah `observation` ↔ `station` funguje perfektne, ale prepojenie pacientov chýba.  


## (B-1b)

In [ ]:
skin_temp = observation["Skin Temperature"]
spo = observation["SpO₂"]
hr =  observation["HR"]
pi = observation["PI"]
rr = observation["RR"]  
prv = observation["PRV"]
bp = observation["BP"]
pvi = observation["PVI"]
sv = observation["SV"]
co = observation["CO"]

In [ ]:
cols = ["Skin Temperature", "SpO₂", "HR", "PI", "RR", "PRV", "BP", "PVI", "SV", "CO"]
observation[cols].describe()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(skin_temp, bins=60, edgecolor="black")
plt.xlabel("Skin Temperature °C")


plt.subplot(1, 2, 2)
plt.boxplot(skin_temp)
plt.xlabel("Skin Temperature °C")

plt.show()


The temperature values are in a range 33-38 °C, mean: 35.9 °C and std: 0.84
This indicates that measurements are within the normal range and do not vary significantly

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(spo, bins=60, edgecolor="black")
plt.xlabel("SpO %")


plt.subplot(1, 2, 2)
plt.boxplot(spo)
plt.xlabel("SpO %")

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(hr, bins=60, edgecolor="black")
plt.xlabel("HR bpm")

plt.subplot(1, 2, 2)
plt.boxplot(hr)
plt.xlabel("HR bpm")

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(pi, bins=60, edgecolor="black")
plt.xlabel("PI %")

plt.subplot(1, 2, 2)
plt.boxplot(pi)
plt.xlabel("PI %")

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(rr, bins=60, edgecolor="black")
plt.xlabel("RR")

plt.subplot(1, 2, 2)
plt.boxplot(rr)
plt.xlabel("RR")

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(prv, bins=60, edgecolor="black")
plt.ylabel("PRV ms")

plt.subplot(1, 2, 2)
plt.boxplot(prv)
plt.xlabel("PRV ms")

plt.show()

Pulse rate variability are in a range 20-200ms, mean: 117.62, std: 21.83
This indicates that the average value is within the normal range, but some values are widely scattered

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(bp, bins=60, edgecolor="black")
plt.xlabel("BP")

plt.subplot(1, 2, 2)
plt.boxplot(bp)
plt.xlabel("BP")

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(pvi, bins=60, edgecolor="black")
plt.xlabel("pvi")

plt.subplot(1, 2, 2)
plt.boxplot(pvi)
plt.xlabel("pvi")

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(sv, bins=60, edgecolor="black")
plt.xlabel("sv")

plt.subplot(1, 2, 2)
plt.boxplot(sv)
plt.xlabel("sv")

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(co, bins=60, edgecolor="black")
plt.xlabel("co")

plt.subplot(1, 2, 2)
plt.boxplot(co)
plt.xlabel("co")

plt.show()

## (С-1)

In [ ]:
print(patient.columns)

In [ ]:
numeric = observation.select_dtypes(include=['number'])

corr = numeric.corr()

plt.figure(figsize=(14,10))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Korelačná matica fyziologických atribútov (observation.csv)")
plt.show()

### Interpretácia výsledkov párovej analýzy dát

Z korelačnej matice (obrázok vyššie) je možné identifikovať viacero významných vzťahov medzi fyziologickými atribútmi:

- **CO a HR (r = 0.76)** – veľmi silná pozitívna korelácia, ktorá zodpovedá očakávanej závislosti medzi srdcovou frekvenciou a srdcovým výdajom.  
- **PVI a Blood Flow Index (r = 0.67)** – silný vzťah medzi variabilitou perfúzie a prietokom krvi.  
- **Oximetry a PVI (r = 0.67)** – saturácia kyslíkom úzko súvisí s variabilitou perfúzie.  
- **Skin Temperature a PI (r = −0.49)** – negatívna korelácia; s rastúcou teplotou kože klesá perfúzny index.  
- **Skin Temperature a Oximetry (r = 0.37)** – mierna pozitívna závislosť, naznačuje možné prepojenie medzi periférnou teplotou a saturáciou.  
- Väčšina ostatných atribútov (napr. `latitude`, `longitude`, `SNR`, `PRV`) nevykazuje štatisticky významné lineárne vzťahy.

Tieto zistenia poukazujú na fyziologické súvislosti medzi vybranými premennými
a pomáhajú určiť, ktoré atribúty môžu byť relevantné pri budúcom modelovaní
a predikcii cieľovej premennej `oximetry`.


In [ ]:
corr_pairs = corr.unstack().sort_values(ascending=False)
corr_pairs = corr_pairs[(corr_pairs < 0.999) & (corr_pairs > -0.999)]
print("🔝 Top 10 korelácií medzi atribútmi:\n")
print(corr_pairs.head(10))

In [ ]:
pairs_to_plot = [
    ("CO", "HR"),
    ("oximetry", "PVI"),
    ("Skin Temperature", "oximetry"),
]

for x, y in pairs_to_plot:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(data=observation, x=x, y=y, alpha=0.6)
    plt.title(f"Vzťah medzi {x} a {y}")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()

### Interpretácia výsledkov

Top 10 korelácií ukazuje, že:

- **CO a HR (r = 0.76)** – silná lineárna závislosť, vyššia srdcová frekvencia znamená väčší srdcový výdaj.  
- **Oximetry a PVI (r = 0.67)** – saturácia kyslíkom úzko súvisí s variabilitou perfúzie.  
- **Skin Temperature a Oximetry (r = 0.37)** – mierna pozitívna korelácia, naznačuje vplyv periférnej teploty na saturáciu.  
- **EtCO₂ a PI (r = 0.31)** – slabší, ale viditeľný pozitívny vzťah.  

Tieto výsledky naznačujú, ktoré premenné môžu mať najväčší význam pri budúcej predikcii
cieľovej premennej `oximetry`.


## (D-1b)

In [ ]:
corr = observation.corr(numeric_only=False)
corr['oximetry'].sort_values()

In [ ]:
sns.scatterplot(data=observation, x='PVI', y='oximetry')
plt.xlabel("PVI %")
plt.ylabel("oximetry")

In [ ]:
sns.scatterplot(data=observation, x='Skin Temperature', y='oximetry')
plt.xlabel("Skin Temperature °C")
plt.ylabel("oximetry")

In [ ]:
sns.scatterplot(data=observation, x='EtCO₂', y='oximetry')
plt.xlabel("EtCO₂ mmHg")
plt.ylabel("oximetry")

## (A-2b)

Check data types

In [ ]:
observation.dtypes

In [ ]:
station.dtypes

In [ ]:
patient.dtypes

Format data types

In [ ]:
station['revision'] = pd.to_datetime(station['revision'], format="mixed")
station['station'] = station['station'].astype('string')
station['QoS'] = station['QoS'].astype('category')
station['location'] = station['location'].astype('string')

In [ ]:
patient['job'] = patient['job'].astype('string')
patient['ssn'] = patient['ssn'].astype('string')
patient['blood_group'] = patient['blood_group'].astype('category')
patient['company'] = patient['company'].astype('string')
patient['name'] = patient['name'].astype('string')
patient['username'] = patient['username'].astype('string')
patient['residence'] = patient['residence'].astype('string')
patient['registration'] = patient['registration'].astype('string')
patient['address'] = patient['address'].astype('string')
patient['mail'] = patient['mail'].astype('string')
patient[['longitude', 'latitude']] = (
    patient['current_location']
    .astype(str)
    .str.extract(r"Decimal\('([\d\.\-]+)'\), Decimal\('([\d\.\-]+)'\)")
    .astype(float)
)
patient.drop(columns=['current_location'], inplace=True)

Check nulls

In [ ]:
observation.isnull().sum()

In [ ]:
station.isnull().sum()

In [ ]:
patient.isnull().sum()

In [ ]:
patient['residence'] = patient['residence'].fillna('Unknown')
patient['job'] = patient['job'].fillna('Unknown')
patient['address'] = patient['address'].fillna('Unknown')
patient['longitude'] = patient['longitude'].fillna(0)
patient['latitude'] = patient['latitude'].fillna(0)

Check duplicates

In [ ]:
observation.duplicated().sum()

In [ ]:
observation = observation.drop_duplicates()

In [ ]:
station.duplicated().sum()

In [ ]:
patient.duplicated().sum()

## (B-2b)

In [ ]:
import re

ranges = pd.read_csv("./094/sensor_variable_range.csv", sep="\t")
print(ranges.head())


def parse_range(r):
    nums = re.findall(r"[\d\.]+", str(r))
    if len(nums) >= 2:
        return float(nums[0]), float(nums[1])
    else:
        return None, None


ranges[["Min", "Max"]] = ranges["Value Range"].apply(lambda r: pd.Series(parse_range(r)))
ranges = ranges.dropna(subset=["Min", "Max"])
ranges.loc[ranges["Variable"] == "BP", ["Min", "Max"]] = [90.0, 120.0]
print(ranges[["Variable", "Min", "Max"]])

In [ ]:
anomalies = []
for _, row in ranges.iterrows():
    var = row["Variable"]
    low, high = row["Min"], row["Max"]
    if var in observation.columns:
        vals = pd.to_numeric(observation[var], errors="coerce")
        invalid_mask = vals.lt(low) | vals.gt(high)
        anomalies.append({
            "Atribút": var,
            "Počet abnormálnych hodnôt": int(invalid_mask.sum()),
            "Min povolené": low,
            "Max povolené": high,
            "Príklady idx": list(invalid_mask[invalid_mask].index[:5]) 
        })

anomalies_df = pd.DataFrame(anomalies).sort_values("Počet abnormálnych hodnôt", ascending=False)
anomalies_df

### B-2b Kontrola správnosti v dátach

Dáta z *observation.csv* boli porovnané s referenčnými rozsahmi fyziologických parametrov zo *sensor_variable_range.csv*.  
V žiadnom z atribútov neboli zistené abnormálne hodnoty mimo definovaných intervalov,  
čo naznačuje, že dataset neobsahuje chybné alebo extrémne merania.  

Pre istotu bola ďalej vykonaná kontrola nelogických kombinácií hodnôt
(vzťahov medzi atribútmi), ktoré by mohli naznačovať chyby senzora alebo anotácie.

In [ ]:
logic_errors = []

# Давление = 0 при наличии пульса — сенсорная ошибка
if "BP" in observation.columns and "HR" in observation.columns:
    mask = (observation["BP"] == 0) & (observation["HR"] > 0)
    logic_errors.append(("BP = 0 a HR > 0", mask.sum()))

# Проверка согласованности сердечного выброса: CO ≈ HR × SV / 1000
if all(col in observation.columns for col in ["CO", "HR", "SV"]):
    co_est = observation["HR"] * observation["SV"] / 1000.0
    mask = (observation["CO"] - co_est).abs() > 0.5 * co_est.fillna(0).abs()
    logic_errors.append(("|CO - HR*SV/1000| > 30%", mask.sum()))

# Высокое качество сигнала, но низкий SNR — нелогично
if "Signal Quality Index" in observation.columns and "SNR" in observation.columns:
    mask = (observation["Signal Quality Index"] >= 80) & (observation["SNR"] < 20)
    logic_errors.append(("Signal Quality Index ≥ 80 a SNR < 20", mask.sum()))

# Низкое качество сигнала, но идеальная SpO₂ — подозрительно
if "Signal Quality Index" in observation.columns and "SpO₂" in observation.columns:
    mask = (observation["Signal Quality Index"] <= 10) & (observation["SpO₂"] >= 99)
    logic_errors.append(("Signal Quality Index ≤ 10 a SpO₂ ≥ 99", mask.sum()))

# Комнатный кислород (FiO₂ ≈ 21%), но низкая SpO₂ — возможная ошибка
if "FiO₂" in observation.columns and "SpO₂" in observation.columns:
    mask = (observation["FiO₂"] <= 22) & (observation["SpO₂"] < 85)
    logic_errors.append(("FiO₂ ≈ 21% a SpO₂ < 85%", mask.sum()))

# Геокоординаты вне допустимого диапазона
if "latitude" in observation.columns and "longitude" in observation.columns:
    mask = (
        observation["latitude"].lt(-90)
        | observation["latitude"].gt(90)
        | observation["longitude"].lt(-180)
        | observation["longitude"].gt(180)
    )
    logic_errors.append(("Latitude/Longitude mimo rozsah", mask.sum()))

# Бонус: RR и EtCO₂ не должны расходиться более чем на порядок
if "RR" in observation.columns and "EtCO₂" in observation.columns:
    mask = (observation["RR"] > 40) & (observation["EtCO₂"] < 20)
    logic_errors.append(("RR > 40 a EtCO₂ < 20", mask.sum()))

logic_df = pd.DataFrame(logic_errors, columns=["Podmienka", "Počet porušení"])
logic_df

### B-2b Kontrola správnosti v dátach

Na základe referenčných rozsahov zo súboru *sensor_variable_range.csv* bola vykonaná kontrola správnosti hodnôt v datasete *observation.csv*.

- Neboli zistené žiadne **abnormálne hodnoty** mimo povolených fyziologických intervalov.
- Následne bola vykonaná aj **kontrola logických vzťahov** medzi atribútmi:
  - `BP = 0 a HR > 0`
  - `|CO – HR×SV/1000| > 30 %`
  - `Signal Quality Index ≥ 80 a SNR < 20`
  - `Signal Quality Index ≤ 10 a SpO₂ ≥ 99`
  - `FiO₂ ≈ 21 % a SpO₂ < 85 %`
  - `Latitude/Longitude mimo rozsah`
  - `RR > 40 a EtCO₂ < 20`

Všetky podmienky mali **0 porušení**, čo znamená, že dataset neobsahuje nelogické alebo chybné kombinácie údajov.  
Dáta sú teda **konzistentné, bez anomálií** a vhodné na ďalšiu fázu projektu – analýzu vzťahov (C-1) a modelovanie cieľovej premennej `oximetry`.

## (C-1b)

In [ ]:
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]

In [ ]:
co_out = identify_outliers(co)
co = co.drop(co_out.index)

pvi_out = identify_outliers(pvi)
pvi = pvi.drop(pvi_out.index)

sv_out = identify_outliers(sv)
sv = sv.drop(sv_out.index)

bp_out = identify_outliers(bp)
bp = bp.drop(bp_out.index)

In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(2, 4, 1)
plt.hist(pvi, bins=60, edgecolor="black")
plt.xlabel("pvi")

plt.subplot(2, 4, 2)
plt.boxplot(pvi)
plt.xlabel("pvi")

plt.subplot(2, 4, 3)
plt.hist(co, bins=60, edgecolor="black")
plt.xlabel("co")

plt.subplot(2, 4, 4)
plt.boxplot(co)
plt.xlabel("co")

plt.subplot(2, 4, 5)
plt.hist(sv, bins=60, edgecolor="black")
plt.xlabel("sv")

plt.subplot(2, 4, 6)
plt.boxplot(sv)
plt.xlabel("sv")

plt.subplot(2, 4, 7)
plt.hist(bp, bins=60, edgecolor="black")
plt.xlabel("BP")

plt.subplot(2, 4, 8)
plt.boxplot(bp)
plt.xlabel("BP")


plt.show()

In [ ]:
def replace_outliers(a):
    lower = a.quantile(0.05)
    upper = a.quantile(0.95)
    
    clipped = a.clip(lower, upper)
    
    return clipped

In [ ]:
prv = replace_outliers(prv)

skin_temp = replace_outliers(skin_temp)

hr = replace_outliers(hr)

spo = replace_outliers(spo)

In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(2, 4, 1)
plt.hist(prv, bins=60, edgecolor="black")
plt.ylabel("PRV ms")

plt.subplot(2, 4, 2)
plt.boxplot(prv)
plt.xlabel("PRV ms")

plt.subplot(2, 4, 3)
plt.hist(skin_temp, bins=60, edgecolor="black")
plt.xlabel("Skin Temperature °C")

plt.subplot(2, 4, 4)
plt.boxplot(skin_temp)
plt.xlabel("Skin Temperature °C")

plt.subplot(2, 4, 5)
plt.hist(hr, bins=60, edgecolor="black")
plt.xlabel("HR bpm")

plt.subplot(2, 4, 6)
plt.boxplot(hr)
plt.xlabel("HR bpm")

plt.subplot(2, 4, 7)
plt.hist(spo, bins=60, edgecolor="black")
plt.xlabel("SpO %")


plt.subplot(2, 4, 8)
plt.boxplot(spo)
plt.xlabel("SpO %")

plt.show()